In [1]:
# %% [markdown]
# # 継続学習手法ごとの Memory / Latency 平均計算
# - ER / EWC / PNN / None の 4手法に対応
# - 各手法ごとに指定した CSV 群を読み込み
# - Memory と Latency の平均を算出して DataFrame にまとめる

# %%
import os
import glob
import pandas as pd

# --- CSVディレクトリの指定 ---
csv_dirs = {
    "PNN": "/work/csv/uci-har/cil_each/pnn",
    "EWC": "/work/csv/uci-har/cil_each/ewc/lambda50000",
    "ER":  "/work/csv/uci-har/cil_each/er",
    "None": "/work/csv/uci-har/cil_each/ncp"
}

# --- 対象カラム名（CSVの列名に合わせて修正してください） ---
MEMORY_COLS = ["推論時 Memory Allocated [MB]", "推論時 Memory Reserved [MB]",
               "訓練時 Memory Allocated [MB]", "訓練時 Memory Reserved [MB]"]
LATENCY_COLS = ["平均レイテンシ (ms/バッチ)", "p95 レイテンシ (ms/バッチ)"]

# %%
def compute_mean_from_csvs(dir_path):
    """指定ディレクトリ内の CSV を全て読み込み、Memory/Latency の平均を返す"""
    csv_files = glob.glob(os.path.join(dir_path, "*.csv"))
    if not csv_files:
        return None

    dfs = []
    for f in csv_files:
        try:
            df = pd.read_csv(f)
            dfs.append(df)
        except Exception as e:
            print(f"読み込み失敗: {f} -> {e}")
    
    if not dfs:
        return None

    df_all = pd.concat(dfs, ignore_index=True)

    results = {}
    for col in MEMORY_COLS + LATENCY_COLS:
        if col in df_all.columns:
            results[col] = df_all[col].mean()
    return results

# %%
# --- 全手法について計算 ---
results_all = {}
for method, dir_path in csv_dirs.items():
    res = compute_mean_from_csvs(dir_path)
    if res:
        results_all[method] = res

# --- DataFrame化 ---
df_summary = pd.DataFrame(results_all).T
df_summary


/tmp/ipykernel_137471/1515785932.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,推論時 Memory Allocated [MB],推論時 Memory Reserved [MB],訓練時 Memory Allocated [MB],訓練時 Memory Reserved [MB],平均レイテンシ (ms/バッチ),p95 レイテンシ (ms/バッチ)
PNN,36.592980,48.560765,NaN,NaN,61.782955,65.501622
EWC,36.787084,48.284954,NaN,NaN,62.302273,66.167556
None,36.743083,48.600043,NaN,NaN,62.229169,67.045908
